In [12]:
import numpy as np
import pandas as pd
import torch
import re
from transformers import AutoModel, AutoTokenizer, RobertaForSequenceClassification

In [14]:
df = pd.read_csv('mongodb_comments.csv')
df = df[['_id','user_id','comment_on','object_id','content','device']]
df.head(5)

C:\Users\nguye\AppData\Local\Temp\ipykernel_7620\3807613248.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('mongodb_comments.csv')


,_id,user_id,comment_on,object_id,content,device
0,{'$oid': '5553269a1a77ce5d65ea0604'},464817,event,54a53e4817dc131729ede7c8,chuong trinh hay qua,android
1,{'$oid': '555327621a77ce5d65ea0605'},28,vod,552b177417dc135c88d7d6ae,hehe không biết chừng nào có tập 6,android
2,{'$oid': '555357f71a77ce5d65ea0606'},424467,vod,551a43b417dc135c58f3c691,"lồng tiếng mà như tập đọc, nghe không nuốt nổi...",web-playfpt
3,{'$oid': '5553f78b1a77ce5d65ea0607'},3839,vod,55532dea17dc130f3beee9c2,phim hay qua,android
4,{'$oid': '555412d91a77ce5d65ea0608'},527201,vod,5453639f17dc1371f538a4cd,hay qua do,android


In [15]:
df.columns

Index(['_id', 'user_id', 'comment_on', 'object_id', 'content', 'device'], dtype='object')

In [19]:
# pre-process data
test = 'Phim đck k không, ko dc đc được hay?'

# dùng regex để standardize
def standardize_data(row):
    # chuyển viết hoa thành viết thường
    row = row.lower()
    # Xóa dấu chấm, phẩy, hỏi ở cuối câu
    row = re.sub(r"[\.,\?]+$", "", row)
    # Xóa tất cả dấu chấm, phẩy, chấm phẩy, chấm thang, ... trong câu
    row = row.replace(",", " ").replace(".", " ") \
        .replace(";", " ").replace("“", " ") \
        .replace(":", " ").replace("”", " ") \
        .replace('"', " ").replace("'", " ") \
        .replace("!", " ").replace("?", " ") \
        .replace("-", " ").replace("?", " ")

    # Standardize từ viết tắt như ko, đc, vn
    row = re.sub("ko? ","không ", row)
    row = re.sub("[dđ]c","được", row)
    row = re.sub("vn","việt nam", row)
    row = re.sub("roi","rồi", row)
    row = re.sub("qua", "quá", row)
    
    # Xóa các space ở đầu và cuối câu
    row = row.strip()
    return row
# standardize câu test print(standardize_data(test))

df['content'] = df['content'].astype(str)
df['content_std'] = df['content'].apply(standardize_data)

df.head(5)

,_id,user_id,comment_on,object_id,content,device,content_std
0,{'$oid': '5553269a1a77ce5d65ea0604'},464817,event,54a53e4817dc131729ede7c8,chuong trinh hay qua,android,chuong trinh hay qua
1,{'$oid': '555327621a77ce5d65ea0605'},28,vod,552b177417dc135c88d7d6ae,hehe không biết chừng nào có tập 6,android,hehe không biết chừng nào có tập 6
2,{'$oid': '555357f71a77ce5d65ea0606'},424467,vod,551a43b417dc135c58f3c691,"lồng tiếng mà như tập đọc, nghe không nuốt nổi...",web-playfpt,lồng tiếng mà như tập đọc nghe không nuốt nổi...
3,{'$oid': '5553f78b1a77ce5d65ea0607'},3839,vod,55532dea17dc130f3beee9c2,phim hay qua,android,phim hay qua
4,{'$oid': '555412d91a77ce5d65ea0608'},527201,vod,5453639f17dc1371f538a4cd,hay qua do,android,hay qua do


In [20]:
# load pre trained model
model = RobertaForSequenceClassification.from_pretrained("wonrax/phobert-base-vietnamese-sentiment")

# load Tokenizer từ model (dùng để tách từ trong câu)
tokenizer = AutoTokenizer.from_pretrained("wonrax/phobert-base-vietnamese-sentiment", use_fast=False)


C:\Users\nguye\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\nguye\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


C:\Users\nguye\anaconda3\Lib\site-packages\transformers\modeling_utils.py:479: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location=

In [30]:
# test câu
sentence = 'chương trình hay quá'

# tách câu thành list từng từ
input_ids = torch.tensor([tokenizer.encode(sentence)])

with torch.no_grad():
    out = model(input_ids)
    # predict output bằng pretrained model
    out = pd.Series(out.logits.softmax(dim=-1).tolist()[0], index= ['NEG', 'POS', 'NEU'])
    print(out)

NEG    0.017476
POS    0.703764
NEU    0.278760
dtype: float64


In [32]:
# hàm để đánh giá từng comment
def sentiment(sentence):
    try:
        input_ids = torch.tensor([tokenizer.encode(sentence)])
    except:
        return 2

    if input_ids.shape[1] > tokenizer.model_max_length:
        return 2

    with torch.no_grad():
        out = model(input_ids)
        return np.argmax(out.logits.softmax(dim=-1).tolist()[0])

print(sentiment(sentence))

1


In [33]:
import time
start_time = time.time()
# test 1000 first comments
test = df[:999]
# apply tất cả comment
test['sentiment'] = test['content_std'].apply(sentiment)

print("--- %s seconds ---" % (time.time() - start_time))
test

Token indices sequence length is longer than the specified maximum sequence length for this model (513 > 256). Running this sequence through the model will result in indexing errors


--- 42.873390913009644 seconds ---


C:\Users\nguye\AppData\Local\Temp\ipykernel_7620\3620050311.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['sentiment'] = test['content_std'].apply(sentiment)


,_id,user_id,comment_on,object_id,content,device,content_std,sentiment
0,{'$oid': '5553269a1a77ce5d65ea0604'},464817,event,54a53e4817dc131729ede7c8,chuong trinh hay qua,android,chuong trinh hay qua,2
1,{'$oid': '555327621a77ce5d65ea0605'},28,vod,552b177417dc135c88d7d6ae,hehe không biết chừng nào có tập 6,android,hehe không biết chừng nào có tập 6,1
2,{'$oid': '555357f71a77ce5d65ea0606'},424467,vod,551a43b417dc135c58f3c691,"lồng tiếng mà như tập đọc, nghe không nuốt nổi...",web-playfpt,lồng tiếng mà như tập đọc nghe không nuốt nổi...,0
3,{'$oid': '5553f78b1a77ce5d65ea0607'},3839,vod,55532dea17dc130f3beee9c2,phim hay qua,android,phim hay qua,1
4,{'$oid': '555412d91a77ce5d65ea0608'},527201,vod,5453639f17dc1371f538a4cd,hay qua do,android,hay qua do,2
...,...,...,...,...,...,...,...,...
994,{'$oid': '55632d3f1a77ce110a495deb'},551293,vod,5549cace17dc132a0cee1801,"có thuyết minh luôn, ủng hộ nhiều",web-playfpt,có thuyết minh luôn ủng hộ nhiều,1
995,{'$oid': '556330541a77ce110a495dec'},551316,vod,552b25d217dc135c8ad7d710,tập 16 bị nhầm sub của phim cặp đôi ngoại cảm ...,android,tập 16 bị nhầm sub của phim cặp đôi ngoại cảm ...,2
996,{'$oid': '556335841a77ce110a495ded'},256139,vod,552b25d217dc135c8ad7d710,Tập 16 bị nhầm sub rùi @@,web-playfpt,tập 16 bị nhầm sub rùi @@,0
997,{'$oid': '556337371a77ce110a495dee'},415657,vod,5552f32517dc130c630e67c9,おいし…私輪 べｊれでえわ,web-playfpt,おいし…私輪 べｊれでえわ,1


In [36]:
# apply tất cả comment
df['sentiment'] = df['content_std'].apply(sentiment)

print("--- %s seconds ---" % (time.time() - start_time))
df

--- 120372.09714794159 seconds ---


,_id,user_id,comment_on,object_id,content,device,content_std,sentiment
0,{'$oid': '5553269a1a77ce5d65ea0604'},464817,event,54a53e4817dc131729ede7c8,chuong trinh hay qua,android,chuong trinh hay qua,2
1,{'$oid': '555327621a77ce5d65ea0605'},28,vod,552b177417dc135c88d7d6ae,hehe không biết chừng nào có tập 6,android,hehe không biết chừng nào có tập 6,1
2,{'$oid': '555357f71a77ce5d65ea0606'},424467,vod,551a43b417dc135c58f3c691,"lồng tiếng mà như tập đọc, nghe không nuốt nổi...",web-playfpt,lồng tiếng mà như tập đọc nghe không nuốt nổi...,0
3,{'$oid': '5553f78b1a77ce5d65ea0607'},3839,vod,55532dea17dc130f3beee9c2,phim hay qua,android,phim hay qua,1
4,{'$oid': '555412d91a77ce5d65ea0608'},527201,vod,5453639f17dc1371f538a4cd,hay qua do,android,hay qua do,2
...,...,...,...,...,...,...,...,...
885164,{'$oid': '629cc3395e052026df2928dd'},19094650,NaN,629c7e1d2089bd01c24f5eb9,"Trước trận chưa nghĩ gì nhiều về chiến thắng, ...",android,trước trận chưa nghĩ gì nhiều về chiến thắng ...,1
885165,{'$oid': '629e14fe5e052026df2928de'},19094650,NaN,629df0b42089bd00568e03e3,đá mấy ngày mới nhớ giải năm nay không có trun...,android,đá mấy ngày mới nhớ giải năm nay không có trun...,0
885166,{'$oid': '62b9ae385e052026df2928df'},16178367,vod,5e8aef312089bd088fbb8f72,Ngôi nhà nhất toạ đang thi đấu nhóm 6 người hì...,ios,ngôi nhà nhất toạ đang thi đấu nhóm 6 người hì...,1
885167,{'$oid': '62ee7c6e5e052026df2928e0'},11866766,NaN,5f951a462089bd06685a7992,"phim hay quá,tuyệt vời cảm ơn fpt",android,phim hay quá tuyệt vời cảm ơn fpt,1


In [38]:
df.to_csv('sentiment.csv', index= False)